## Burned area delineation

In [1]:
workflow = dict([('id', 'burned-area-delineation'),
                ('label', 'Burned area delineation'),
                ('doc', 'Burned area delineation using two techniques')])


In [2]:
pre_event = dict([('id', 'pre_event'), 
                  ('label', 'Pre-event product for burned area delineation'),
                  ('doc', 'Pre-event product for burned area delineation'),
                  ('value', '/tmp/mql_19ii/'), 
                  ('type', 'Directory')])

In [3]:
post_event = dict([('id', 'post_event'), 
                  ('label', 'Post-event product for burned area delineation'),
                  ('doc', 'Post-event product for burned area delineation'),
                  ('value', '/tmp/j9o12gpe/'), 
                  ('type', 'Directory')])

In [4]:
ndvi_threshold = dict([('id', 'ndvi_threshold'),
                       ('value', '0.19'),
                       ('label', 'NDVI difference threshold'),
                       ('doc', 'NDVI difference threshold'),
                       ('type', 'string')]) 

In [21]:
ndwi_threshold = dict([('id', 'ndwi_threshold'),
                       ('value', '0.18'),
                       ('label', 'NDWI difference threshold'),
                       ('doc', 'NDWI difference threshold'),
                       ('type', 'string')]) 

In [22]:
store_apikey = dict([('id', ''),
                       ('value', 'Ufjh10ywUU'),
                       ('label', 'Store API key'),
                       ('doc', 'Store API key'),
                       ('type', 'string')]) 

In [23]:
store_username = dict([('id', ''),
                       ('value', 'eoepca-demo-storage'),
                       ('label', 'Store username'),
                       ('doc', 'Store username'),
                       ('type', 'string')]) 

In [24]:
username = None if store_username['value'] == '' else store_username['value']
api_key = None if store_apikey['value'] == '' else store_apikey['value']

In [25]:
import os
import sys
import gdal
import numpy as np
import logging
from pystac import *
from time import sleep
from helpers import *
import shutil

gdal.UseExceptions()

if not 'PREFIX' in os.environ.keys():
    
    os.environ['PREFIX'] = '/opt/anaconda/envs/env_burned_area_delineation'

os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')

In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
pre_event_cat = Catalog.from_file(os.path.join(pre_event['value'], 'catalog.json'))
post_event_cat = Catalog.from_file(os.path.join(post_event['value'], 'catalog.json'))

In [28]:
pre_event_cat.describe()

* <Catalog id=catalog>
  * <Item id=INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805>


In [29]:
post_event_cat.describe()

* <Catalog id=catalog>
  * <Item id=INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805>


In [30]:
pre_event_item = next(pre_event_cat.get_items())
post_event_item = next(post_event_cat.get_items())

In [31]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr = get_assets(pre_event_item)
asset_post_ndvi, asset_post_ndwi, asset_post_nbr = get_assets(post_event_item)

In [32]:
asset_pre_ndvi, asset_pre_ndwi, asset_pre_nbr

('https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/j2yo5b37/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NDVI_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif',
 'https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/j2yo5b37/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NDWI_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif',
 'https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/j2yo5b37/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NBR_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif')

In [33]:
asset_post_ndvi, asset_post_ndwi, asset_post_nbr

('https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/iddysyz9/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NDVI_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif',
 'https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/iddysyz9/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NDWI_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif',
 'https://nx10438.your-storageshare.de/remote.php/dav/files/eoepca-demo-storage/wf-xwz-vegetation-index/iddysyz9/INDEX_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805/NBR_S2B_MSIL2A_20201006T004709_N0214_R102_T53HPA_20201006T022805.tif')

In [34]:
scaling_factor = 1/10000

scaling_factor

0.0001

If NDWI i2 - NDWI i1 > 0.18 and If NDVI i2 - NDVI i1 > 0.19 then burned pixels

In [35]:
_mem = '/vsimem/mem.tif'

Process NDVI difference

In [36]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_ndvi, username, api_key),
                         outputType=gdal.GDT_Int16)

width = temp_ds.RasterXSize
height = temp_ds.RasterYSize
geo_transform = temp_ds.GetGeoTransform()
geo_ref = temp_ds.GetProjectionRef()

pre_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [37]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_post_ndvi, username, api_key),
                         outputType=gdal.GDT_Int16)

post_ndvi = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_ndvi = ((pre_ndvi - post_ndvi) * scaling_factor).astype(float)

pre_ndvi = post_ndvi = None

Process NDWI difference

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_ndwi, username, api_key),
                         outputType=gdal.GDT_Int16)

pre_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_post_ndwi, username, api_key),
                         outputType=gdal.GDT_Int16)

post_ndwi = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_ndwi = ((pre_ndwi - post_ndwi) * scaling_factor).astype(float)

pre_ndwi = post_ndwi = None

Burned area delineation

In [ ]:
burned = np.where(((delta_ndwi  > float(ndwi_threshold['value'])) & (delta_ndvi > float(ndvi_threshold['value']))), 1, 0) 

In [ ]:
output_name = 'DELINEATION'

In [ ]:
default_bands = [{'name': 'dnbr',
                  'common_name': 'dNBR'}, 
                 {'name': 'tvi',
                  'common_name': 'tvi'}]

In [ ]:
temp_name = '_BURNED_NDVI_NDWI_THRESHOLD.tif'
output_name = 'BURNED_NDVI_NDWI_THRESHOLD.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Byte)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(burned),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [ ]:
results = {}

results['B01'] = output_name


relativized burn ratio (RBR) 

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_pre_nbr, username, api_key),
                         outputType=gdal.GDT_Int16)

pre_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
temp_ds = gdal.Translate(_mem,
                         get_gdal_asset_path(asset_post_nbr, username, api_key),
                         outputType=gdal.GDT_Int16)

post_nbr = temp_ds.ReadAsArray()

temp_ds = None

In [ ]:
delta_nbr = ((pre_nbr  - post_nbr) * scaling_factor).astype(float)

post_nbr = None

In [ ]:
rbr = delta_nbr / (pre_nbr * scaling_factor + 1.001)

delta_nbr = pre_nbr = None

In [ ]:
temp_name = '_BURNED_RBR.tif'
output_name = 'BURNED_RBR.tif'

driver = gdal.GetDriverByName('GTiff')

output = driver.Create(temp_name, 
                       width, 
                       height, 
                       1, 
                       gdal.GDT_Float32)

output.SetGeoTransform(geo_transform)
output.SetProjection(geo_ref)
output.GetRasterBand(1).WriteArray(rbr),

output.FlushCache()

sleep(5)

output = None

del(output)

cog(temp_name, output_name)

In [ ]:
results['B02'] = output_name

In [ ]:
catalog = Catalog(id='catalog',
                      description='Results')

catalog.clear_items()
catalog.clear_children()

In [ ]:
item_name = 'BURNED_AREA_DELINEATION'


pre_event_item.properties.pop('eo:bands', None)

result_item = Item(id=item_name,
                   geometry=pre_event_item.geometry,
                   bbox=pre_event_item.bbox,
                   datetime=pre_event_item.datetime,
                   properties=pre_event_item.properties)

result_item.common_metadata.set_gsd(10)

eo_item = extensions.eo.EOItemExt(result_item)

In [ ]:
results

In [ ]:
default_bands = {'B01':  {'name': 'NDVI/NDWI Threshold', 'common_name': 'tvi'},
                 'B02': {'name': 'Normalized Burn Ratio (NBR) ', 'common_name': 'dnbr'}}

In [ ]:
bands = []
   
os.makedirs(os.path.join('.', item_name),
                exist_ok=True)

for key, value in results.items():

    print(key, value)
    
    

    result_item.add_asset(key=key,
                          asset=Asset(href='./{}'.format(value), 
                                    media_type=MediaType.GEOTIFF))

    asset = result_item.get_assets()[key]                                   

    stac_band = extensions.eo.Band.create(name=key, 
                                           common_name=default_bands[key]['common_name'],
                                                description=default_bands[key]['name'])
    bands.append(stac_band)

    eo_item.set_bands([stac_band], asset=asset)

    shutil.move(value, os.path.join('.', item_name, value))
    
eo_item.set_bands(bands)

eo_item.apply(bands)  

### Results verification

In [ ]:
catalog.add_items([result_item])
    
catalog.normalize_and_save(root_href='./',
                           catalog_type=CatalogType.SELF_CONTAINED)

In [ ]:
catalog.describe()

In [ ]:
item = next(catalog.get_items())

In [ ]:
item.get_assets()

In [ ]:
asset = item.get_assets()['B01']
asset.get_absolute_href()

In [ ]:
asset = item.get_assets()['B02']
asset.get_absolute_href()

In [ ]:
eo_item = extensions.eo.EOItemExt(item)
    
for index, band in enumerate(eo_item.bands):

    if band.common_name in ['tvi']:

        print(band.common_name, item.assets[band.name].get_absolute_href())
        
    if band.common_name in ['dnbr']:

        print(band.common_name, item.assets[band.name].get_absolute_href())